# Sistemas Inteligentes 2020/2021

## Mini-projeto 2: Puzzle dos quadrados de fósforos - heurísticas



<img src="fosforos.gif" alt="Drawing" style="width: 100px;"/>

#### Entrega: 31 de Março às 23h59

**Nota:** Todas as dúvidas relacionadas com este projeto devem ser dirigidas ao Prof. Nuno Garcia.

**Importante:** Existem dois ficheiros novos no zip: search_v3.py e utils_v2.py . Estes ficheiros são versões mais eficientes que os distribuidos anteriormente, e devem ser os ficheiros de acompanhamento a usar na resolução deste projeto.

### Introdução

Considere o problema do puzzle dos quadrados descrito no enunciado do Mini-projeto 1.

<img src="puzzle_fosforos.png" alt="Drawing" style="width: 550px;"/>

### Formulação

Considere que cada fósforo tem associado um ID. Por exemplo, a configuração inicial (Fig. 1) é composta por todos os fósforos: a primeira linha tem os fósforos ID 1, 2 e 3, a primeira coluna tem os fósforos ID 4, 5, 6, 7, a segunda linha tem os fósforos 8, 9 e 10, e assim consecutivamente.

<img src="fosforo-id.png" alt="Drawing" style="width: 250px;"/>




A representação dos estados é definida pelo conjunto de IDs dos fósforos usados para construir os quadrados.

Podem haver quadrados de 1, 2 e 3 fósforos de largura.

In [ ]:
# quadrados possiveis de lado 1, 2, 3
quad_side_1 = [set([1,4,5,8]), set([2,5,6,9]), set([3,6,7,10]), 
     set([8,11,12,15]), set([9,12,13,16]), set([10,13,14,17]), 
     set([15,18,19,22]), set([16,19,20,23]), set([17,20,21,24])]

quad_side_2 = [set([1,2,4,6,11,13,15,16]), 
     set([2,3,5,7,12,14,16,17]),
     set([8,9,11,13,18,20,22,23]),
     set([9,10,12,14,19,21,23,24])]

quad_side_3 = set([1,2,3,4,7,11,14,18,21,22,23,24])

### Implementação da classe

A classe ProblemaFosforos diz respeito à Parte 2 do trabalho anterior.

In [ ]:
from copy import copy
import random
import timeit
from search_v3.py import *

class ProblemaFosforos(Problem):
    quadrados = [quad_side_3]
    quadrados.extend(quad_side_2)
    quadrados.extend(quad_side_1)

    #hor = [1,2,3,8,9,10,15,16,17,22,23,24]
    #ver = [4,5,6,7,11,12,13,14,18,19,20,21]
    def __init__(self, initial, goal=2):
        self.initial = tuple(initial)
        self.goal = goal

    def actions(self, estado):
        """ Cada acção representa a remoção de um fósforo. Existem 24 acções no total.
         Por exemplo, a acção 1 corresponde a remover o fósforo 1.

         Na situação em que não há nenhum quadrado formado, não adianta realizar mais ações -> []

        Args:
            estado ([type]): [description]

        Returns:
            [type]: [description]
        """

        if len(self.quadrados_validos(estado)) == 0:
            return []
        return list(estado)

    def result(self, estado, accao):
        """Remove o fósforo com id=acçao do estado e retorna um novo estado.

        Args:
            estado (set[int]): conjunto de IDs de fósforos
            accao (int): ID do fósforo a remover

        Returns:
            tuple[int]: novo estado
        """
        s = copy(list(estado))
        s.remove(accao)
        return tuple(s)

    def goal_test(self, estado):
        """Testa se o estado é o estado final.
        O objetivo é ter apenas self.goal quadrados válidos e nenhum fósforo solto.

        Args:
            estado (tuple[int]): estado a ser avaliado.

        Returns:
            bool: se é o estado final.
        """
        qval = self.quadrados_validos(estado)
        soltos = self.fosforos_soltos(estado)
        return len(qval) == self.goal and soltos == 0

    def quadrados_validos(self, estado):
        """Verifica quantos são quadrados válidos estão presentes no estado, usando a informação estática da classe.

        Args:
            estado (tuple[int]): estado a ser avaliado.

        Returns:
            list: lista com os IDs dos fósforos de quadrados válidos
        """
        qval = []
        for i in self.quadrados:
            if i.intersection(set(estado)) == i:
                qval.append(list(i))
        return qval

    def fosforos_soltos(self, estado):
        """Devolve o número de fósforos soltos do estado.

        Args:
            estado (tuple[int]): estado a ser avaliado.

        Returns:
            int: numero de fósforos soltos.
        """
        qval = self.quadrados_validos(estado)
        qval = set([item for sublist in qval for item in sublist])
        return len(set(estado) - qval)

    def display(self, estado):
        """Imprime uma visualização do estado

        Args:
            estado (tuple[int]): estado a ser avaliado.
        """
        N = 3
        for i in range(1, N + 2):
            h = [2 * (i - 1) * N + i + j
                 for j in range(0, N)]  # fosforos horizontais
            v = [(2 * i - 1) * N + i + j
                 for j in range(0, N + 1)]  # fosforos verticais
            for k in h:
                if k in estado:
                    print('  -- ', end='')
                else:
                    print('     ', end='')
            print()
            for k in v:
                if k in estado:
                    print('|    ', end='')
                else:
                    print('     ', end='')
            print()
            
    def h1(self, no):
        """ heurística a ser usada pelos algoritmos de procura informada.
        Args:
            no (Node): objeto da classe Node, definida na search_v3.py
        Returns:
            number: valor_da_heuristica do nó 
        """
        pass  
        # return valor_da_heuristica

Instanciação do problema com o estado inicial da Figura 1.

In [ ]:
x = [i for i in range(1,25)]
p1 = ProblemaFosforos(initial=x)

Função auxiliar ```exec```

In [ ]:
def exec(p, estado, accoes):
    """ Executa uma sequência de acções a partir do estado
        devolve um par (estado, custo)
    """
    custo = 0
    for a in accoes:
        seg = p.result(estado, a)
        custo = p.path_cost(custo, estado, a, seg)
        estado = seg
    p.display(estado)
    print('Custo:', custo)
    print('Goal?', p.goal_test(estado))
    return (estado, custo)

### Utilizar os métodos de procura

Os métodos de procura estão implementados no ficheiro search_v3.py

Por exemplo, a pesquisa em largura em grafo pode ser testada da seguinte forma.

Repare que a procura em largura é igual à procura Greedy se a condição para escolher o próximo nó for a profundidade.

In [ ]:
x = [i for i in range(1, 25)]
p1 = ProblemaFosforos(initial=x)
start = timeit.default_timer()
res = breadth_first_graph_search(p1)
# res = astar_search(p1, p1.h1)
stop = timeit.default_timer()
timeGraph = stop - start
print('Time: ', timeGraph)
print("Solução:", res.solution())
exec(p1, p1.initial, res.solution())

## Exercícios

1. Considerando a formulação descrita anteriormente, qual é o tamanho do espaço de estados?
2. Aplique o algoritmo de procura em largura-primeiro. A que profundidade se encontra a solução encontrada pela procura em largura-primeiro? É a solução óptima?
3. Defina, explique, e implemente duas heurísticas. Elas devem ser substancialmente diferentes. Por exemplo, se uma heurística for H(node) = x, a outra não pode ser H(node)=2\*x, ou H(node)=2\+x. Cada heurística deve ser um método na classe ProblemaFosforos, à semelhança do ```h1```
4. Comente e justifique se as heurísticas são ou não admissiveis.
5. Implemente o algoritmo de aprofundamento progressivo A\*. Este algoritmo utiliza a procura com A\* com profundidade limitada, e repete a procura com profundidades crescentes até encontrar uma solução ou chegar ao limite máximo definido. É semelhante ao algoritmo de aprofundamento progressivo visto nas aulas, mas o algoritmo de procura usado é o  A\*.
    1. Deve fazer o método ```aprofundamento_prog_astar```.
    2. Sugestão: pode utilizar este método para chamar outro, por exemplo uma variante do ```best_first_graph_search```.



In [ ]:
def aprofundamento_prog_astar(problem, h=None, max_depth=50):
    """
    problem: o problema
    h: a heurística
    max_depth: a profundidade máxima permitida
    """
    pass

6. Construa uma [tabela](https://docs.github.com/en/github/writing-on-github/organizing-information-with-tables) em que compare o tempo, nós expandidos, nós visitados, e custo da solução para os algoritmos:
    1. Procura em largura-primeiro.
    2. Procura de custo uniforme.
    2. Greedy com as duas heurísticas propostas.
    3. A\* com as duas heurísticas propostas.
    4. Aprofundamento progressivo com A\* com as duas heurísticas propostas.
    
Sugestão: pode ser útil usar o parâmetro ```display``` das funções de procura.


### Relatório 

O relatório é **obrigatório** e deverá ser apresentado em Jupyter Notebook **(*.ipynb)** e em **.html**, contendo as respostas aos exercícios e o código desenvolvido. 

Qualquer trabalho que não tenha relatório ou não o tenha no formato pedido não será avaliado (i.e., nota 0). É fornecido um ficheiro esqueleto, **SI-proj2-XX.ipynb** (onde XX é o número do grupo) para realização do relatório. Não se esqueça de preencher os nomes e números dos elementos do grupo.

### Entrega

Deverá entregar um **zip** chamado **SI-proj2-XX** (onde XX é o número do grupo) com os dois ficheiros pedidos (relatório em formato `.ipynb` e `.html`). 

*Nota: para gerar a versão html do relatório faça `File -> Download as`.*

**Data limite:** 31 de Março às 23h59